## Note 
This notebook loads the dataset (images already cropped and best patches kept) and trains the model to detect buildings changes.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!ls -al drive/My\ Drive/data2

In [ ]:
train_imgs = np.load("drive/My Drive/data2/imgs_final_dataset.npy")
train_labels = np.load("drive/My Drive/data2/labels_final_dataset.npy")

In [ ]:
train_imgs.shape, train_labels.shape

In [ ]:
plt.imshow(train_labels[300,:,:,0], cmap="gray")

In [ ]:
plt.imshow(train_labels[900,:,:,0], cmap="gray")

In [ ]:
a=train_labels[300,:,:,0]
a[a>0]=1
b=train_labels[900,:,:,0]
c=a-b
c[c==-1]=0

In [ ]:
plt.imshow(c, cmap="gray")

### Model

In [ ]:
img_size = 200
img_input = tf.keras.layers.Input(shape=(img_size, img_size, 6)) # classical unet

#encoder
conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = layers.Dropout(0.2)(conv1)
conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2))(conv1) # classical unet

conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = layers.Dropout(0.2)(conv2)
conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = layers.MaxPooling2D((2, 2))(conv2) # classical unet

conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = layers.Dropout(0.2)(conv3)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
#pool3 = layers.MaxPooling2D((2, 2))(conv3) # classical unet

#bottleneck
#convl = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
#convl = layers.Dropout(0.2)(convl)
#convl = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(convl)

#decoder
#up0 = layers.concatenate([layers.UpSampling2D((2, 2))(convl), conv3], axis=-1)  # classical unet
#conv4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up0)
#conv4 = layers.Dropout(0.2)(conv4)
#conv4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)

#up1 = layers.concatenate([layers.UpSampling2D((2, 2))(conv4), conv2], axis=-1)  # classical unet
up1 = layers.concatenate([layers.UpSampling2D((2, 2))(conv3), conv2], axis=-1)  # classical unet
conv5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv5 = layers.Dropout(0.2)(conv5)
conv5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

up2 = layers.concatenate([layers.UpSampling2D((2, 2))(conv5), conv1], axis=-1)  # classical unet
conv6 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv6 = layers.Dropout(0.2)(conv6)
conv6 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv6)

out = layers.Conv2D( 1, (1, 1) , activation='relu', padding='same')(conv6)
out = layers.Reshape((img_size, img_size, 1))(out) # classical unet

model = models.Model(inputs=img_input, outputs=out)
#model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy")

In [ ]:
model.fit(x = train_imgs[:1400], y = train_labels[:1400], validation_data=(train_imgs[1400:1600], train_labels[1400:1600]), epochs=20)

In [ ]:
model.save("drive/My Drive/data2/model_weights")

In [ ]:
model = tf.keras.models.load_model("drive/My Drive/data2/model_weights")

In [ ]:
res = model.predict(train_imgs)

In [ ]:
number = 4
plt.figure(figsize=(18,8))

indices = [1500,1650,1750,1820]
for index in range(number):
    # display original
    ax = plt.subplot(2, number, index + 1)
    plt.imshow(train_labels[indices[index],:,:,0], cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, number, index + 1 + number)
    plt.imshow(res[indices[index],:,:,0], cmap="gray")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
plt.imshow(train_imgs[400,:,:,3:], cmap="gray")

In [ ]:
plt.imshow(train_labels[1700,:,:,0], cmap="gray")

In [ ]:
plt.imshow(res[1700, :,:,0], cmap="gray")